In [1]:
from msu_esd import Pipe
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

# Define known constants
K1, K2, K3, K4 = 4.5, 4.5, 4.5, 10  # Loss coefficients

rho = 48.05/32.174  # In slugs per cubic feet
mu = 3.715/(3600*32.174)  # In slugs per (ft s) or lbf*s per ft squared
epsilon = 0.0005  # In ft

D20, D12, D10, D8 = np.array([18.812, 11.938, 10.02, 7.981])/12  # Diameters in ft

Ws = 145  # in ft

p1 = Pipe(D20, 2840, epsilon, rho, mu, K=1.78, C=8)
p2 = Pipe(D10, 2380, epsilon, rho, mu, K=5.53, C=8)
p3 = Pipe(D12, 1300, epsilon, rho, mu, K=1.78, C=8)
p4 = Pipe(D10, 1630, epsilon, rho, mu, K=5.53, C=8)
p5 = Pipe(D12, 3000, epsilon, rho, mu, K=3.28, C=8)
p6 = Pipe(D12, 5000, epsilon, rho, mu, K=3.28, C=8)
p7 = Pipe(D12, 1580, epsilon, rho, mu, K=1.78, C=8)
p8 = Pipe(D8, 1550, epsilon, rho, mu, K=5.53, C=8)
p9 = Pipe(D8, 1550, epsilon, rho, mu, K=7.03, C=8)
p10 = Pipe(D10, 5130, epsilon, rho, mu, K=7.78, C=8)
p11 = Pipe(D12, 1875, epsilon, rho, mu, K=2.53, C=8)

def balanced(x, b2, b4, b7, b10):
    Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Q11 = x
    # all expressions need to be set to zero
    return [
        Q1 - Q2 - Q3,
        Q6 - Q2 - Q4,
        Q3 - Q4 - Q5,
        Q5 - Q10 - Q7,
        Q7 - Q8 - Q9,
        Q11 - Q6 - Q7,
        K1*Q2*abs(Q2) + p2.h(Q2) + p6.h(Q6) + p11.h(Q11) + p1.h(Q1) - Ws + 0.1*Q1*abs(Q1) - b2,
        p4.h(Q4) + K3*Q4*abs(Q4) - p2.h(Q2) - K1*Q2*abs(Q2) + p3.h(Q3) + b2 - b4,
        p7.h(Q7) + p8.h(Q8) + K4*Q8*abs(Q8) - p6.h(Q6) - p4.h(Q4) - K3*Q4*abs(Q4) + p5.h(Q5) + b4 - b7,
        p9.h(Q9) + K4*Q9*abs(Q9) - p8.h(Q8) - K4*Q8*abs(Q8),
        p10.h(Q10) + K2*Q10*abs(Q10) - p11.h(Q11) - p9.h(Q9) - K4*Q9*abs(Q9) - p7.h(Q7) + b7 - b10
    ]

def test(pipes_, flow_rates):
    lines = range(1, len(pipes_) + 1)
    Q_values, V_values = [], []
    AHU_1_3, AHU_4 = [], []
    V_less = []

    for pipe_, Q_, i in zip(pipes_, flow_rates, lines):
        Q_values.append(Q_)
        V_ = pipe_.V(Q_)
        V_values.append(V_)

        if V_ < 9:
            V_less.append('Yes')
        else:
            V_less.append('No')

        if i in [2, 4, 10]:
            if 2.713 <= Q_ < 2.808:
                AHU_1_3.append('Yes')
            else:
                AHU_1_3.append('No')
        else:
            AHU_1_3.append('-')

        if i in [8, 9]:
            if 1.272 <= Q_ < 1.316:
                AHU_4.append('Yes')
            else:
                AHU_4.append('No')
        else:
            AHU_4.append('-')

    return pd.DataFrame({'Pipe': lines, 'Q': Q_values, 'V': V_values, 'V<9': V_less, '2.713<=Q<2.808': AHU_1_3, r'1.272<=Q<1.316': AHU_4})

Q_guess = np.array([5, 1, 4, 2, 2, 3, 1, 3, -2, 1, 4])  # This does satisfy mass conservation
b = 67.65, 101.95, 55.7, 44.9
balanced_solution = fsolve(balanced, Q_guess, args=(*b, ))
pipes = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11]
iteration_test = test(pipes, balanced_solution)
iteration_test

,Pipe,Q,V,V<9,2.713<=Q<2.808,1.272<=Q<1.316
0,1,10.692539,5.539665,Yes,-,-
1,2,2.713009,4.954369,Yes,Yes,-
2,3,7.979530,10.265658,No,-,-
3,4,2.713247,4.954805,Yes,Yes,-
4,5,5.266283,6.775068,Yes,-,-
5,6,5.426256,6.980874,Yes,-,-
6,7,2.552338,3.283581,Yes,-,-
7,8,1.279845,3.683965,Yes,-,Yes
8,9,1.272493,3.662802,Yes,-,Yes
9,10,2.713944,4.956078,Yes,Yes,-


In [2]:
# Getting a table of values to choose from
nps = np.array([5, 6, 8, 10, 12, 14, 16, 18, 20, 24, 32, 34, 36, 42])  # Nominal Pipe Size (in)
od = np.array([5.563, 6.625, 8.625, 10.75, 12.75, 14, 16, 18, 20, 24, 32, 34, 36, 42])  # Outer Diameter (in)
t = np.array([0.258, 0.28, 0.322, 0.365, 0.406, 0.438, 0.5, 0.562, 0.594, 0.688, 0.688, 0.688, 0.75, 0.75])  # Thickness (in)
id_ = od - 2*t  # Inner Diameter (in)
id_ft = id_/12

df = pd.DataFrame({'NPS': nps, 'Outer Diameter (in)': od, 'Thickness (in)': t, 'Inner Diameter (in)': id_, 'Inner Diameter (ft)': id_ft})
df

,NPS,Outer Diameter (in),Thickness (in),Inner Diameter (in),Inner Diameter (ft)
0,5,5.563,0.258,5.047,0.420583
1,6,6.625,0.280,6.065,0.505417
2,8,8.625,0.322,7.981,0.665083
3,10,10.750,0.365,10.020,0.835000
4,12,12.750,0.406,11.938,0.994833
5,14,14.000,0.438,13.124,1.093667
6,16,16.000,0.500,15.000,1.250000
7,18,18.000,0.562,16.876,1.406333
8,20,20.000,0.594,18.812,1.567667
9,24,24.000,0.688,22.624,1.885333
